In [ ]:
import pandas as pd

data = "ml-1m"

df = pd.read_csv(f"./processed/{data}/{data}.inter", sep="\t")
df.head()

In [ ]:
rate_size_dic_u = df.groupby("user_id:token").size()
rate_size_dic_i = df.groupby("item_id:token").size()
rate_size_dic_u

In [ ]:
choosed_index_del_u = rate_size_dic_u[rate_size_dic_u < 5].index
choosed_index_del_i = rate_size_dic_i[rate_size_dic_i < 5].index
choosed_index_del_u

In [ ]:
core_df = df[~df["item_id:token"].isin(list(choosed_index_del_i))]
core_df = core_df[~df["user_id:token"].isin(list(choosed_index_del_u))]

len(df), len(core_df)

In [ ]:
core_df.to_csv(f"./processed/{data}/{data}.inter", sep="\t", index=False)